필사해야 하는 부분 : AudioLDMPipeline 클래스의 bind_foward_double_loss()함수.

 (첨부 깃허브 링크 기준 code line 868-1071을 필사해주시면 됩니다.)

https://github.com/yzxing87/Seeing-and-Hearing/blob/main/v2a/audioldm/pipelines/pipeline_audioldm.py

In [ ]:
 def bind_forward_double_loss(
        self,
        prompt: Union[str, List[str]] = None,
        audio_length_in_s: Optional[float] = None,
        num_inference_steps: int = 10,
        guidance_scale: float = 2.5,
        learning_rate: float = 0.1,
        negative_prompt: Optional[Union[str, List[str]]] = None,
        num_waveforms_per_prompt: Optional[int] = 1,
        clip_duration: float = 2.0,
        clips_per_video: int = 5,
        num_optimization_steps: int = 1,
        optimization_starting_point: float = 0.2,
        eta: float = 0.0,
        video_paths: Union[str, List[str]] = None,
        generator: Optional[Union[torch.Generator, List[torch.Generator]]] = None,
        latents: Optional[torch.FloatTensor] = None,
        prompt_embeds: Optional[torch.FloatTensor] = None,
        negative_prompt_embeds: Optional[torch.FloatTensor] = None,
        return_dict: bool = True,
        callback: Optional[Callable[[int, int, torch.FloatTensor], None]] = None,
        callback_steps: Optional[int] = 1,
        cross_attention_kwargs: Optional[Dict[str, Any]] = None,
        output_type: Optional[str] = "np",
    ):
        # 0. Convert audio input length from seconds to spectrogram height
        vocoder_upsample_factor = np.prod(self.vocoder.config.upsample_rates) / self.vocoder.config.sampling_rate

        if audio_length_in_s is None:
            audio_length_in_s = self.unet.config.sample_size * self.vae_scale_factor * vocoder_upsample_factor

        height = int(audio_length_in_s / vocoder_upsample_factor)

        original_waveform_length = int(audio_length_in_s * self.vocoder.config.sampling_rate)
        if height % self.vae_scale_factor != 0:
            height = int(np.ceil(height / self.vae_scale_factor)) * self.vae_scale_factor
            logger.info(
                f"Audio length in seconds {audio_length_in_s} is increased to {height * vocoder_upsample_factor} "
                f"so that it can be handled by the model. It will be cut to {audio_length_in_s} after the "
                f"denoising process."
            )

        # 1. Check inputs. Raise error if not correct
        self.check_inputs(
            prompt,
            audio_length_in_s,
            vocoder_upsample_factor,
            callback_steps,
            negative_prompt,
            prompt_embeds,
            negative_prompt_embeds,
        )

        # 2. Define call parameters
        if prompt is not None and isinstance(prompt, str):
            batch_size = 1
        elif prompt is not None and isinstance(prompt, list):
            batch_size = len(prompt)
        else:
            batch_size = prompt_embeds.shape[0]

        device = self._execution_device
        # here `guidance_scale` is defined analog to the guidance weight `w` of equation (2)
        # of the Imagen paper: https://arxiv.org/pdf/2205.11487.pdf . `guidance_scale = 1`
        # corresponds to doing no classifier free guidance.
        do_classifier_free_guidance = guidance_scale > 1.0

        # 3. Encode input prompt
        # 프롬프트 인코딩
        # Joint-VA, 조건부 생성에 사용됨
        prompt_embeds = self._encode_prompt(
            prompt,
            device,
            num_waveforms_per_prompt,
            do_classifier_free_guidance,
            negative_prompt,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
        )

        # 4. Prepare timesteps
        self.scheduler.set_timesteps(num_inference_steps, device=device)
        timesteps = self.scheduler.timesteps

        # 5. Prepare latent variables
        num_channels_latents = self.unet.config.in_channels
        latents = self.prepare_latents(
            batch_size * num_waveforms_per_prompt,
            num_channels_latents,
            height,
            prompt_embeds.dtype,
            device,
            generator,
            latents,
        )
        latents_dtype = latents.dtype

        # 6. Prepare extra step kwargs
        extra_step_kwargs = self.prepare_extra_step_kwargs(generator, eta)

        # ImageBind 모델을 위한 비디오 데이터 준비
        image_bind_video_input = load_and_transform_video_data(video_paths, device, clip_duration=clip_duration, clips_per_video=clips_per_video, n_samples_per_clip=2)

        # ImageBind 모델 로드 및 설정
        bind_model = imagebind_model.imagebind_huge(pretrained=True)

        bind_model.eval()
        bind_model.to(device)

        for p in bind_model.parameters():
            p.requires_grad = False # 학습 없이 정렬기로만 사용됨


        # 7. Denoising loop
        num_warmup_steps = len(timesteps) - num_inference_steps * self.scheduler.order
        num_warmup_steps_bind = int(len(timesteps) * optimization_starting_point)
        with self.progress_bar(total=num_inference_steps) as progress_bar:
            for i, t in enumerate(timesteps):
                # expand the latents if we are doing classifier free guidance
                latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents

                latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)

                # predict the noise residual
                with torch.no_grad():
                    noise_pred = self.unet(
                        latent_model_input,
                        t,
                        encoder_hidden_states=None,
                        class_labels=prompt_embeds,
                        cross_attention_kwargs=cross_attention_kwargs,
                    ).sample.to(dtype=latents_dtype)

                # perform guidance
                if do_classifier_free_guidance:
                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

                # compute the previous noisy sample x_t -> x_t-1
                latents = self.scheduler.step(noise_pred, t, latents, **extra_step_kwargs).prev_sample
                # 확산 잠재 정렬기 적용
                latents_temp = latents.detach()
                latents_temp.requires_grad = True # 잠재 변수 z_t 최적화 위해서 기울기 필요

                optimizer = torch.optim.Adam([latents_temp], lr=learning_rate)

                if i > num_warmup_steps_bind: # 워밍업 단계 K 이후에만 정렬/가이던스 적용
                    for optim_step in range(num_optimization_steps):
                        with torch.autograd.set_detect_anomaly(True):
                            # 1. compute x0
                            # 노이즈 예측을 사용하여 z_t에서 깨끗한 잠재 변수 z_0을 예측
                            x0 = 1/(self.scheduler.alphas_cumprod[t] ** 0.5) * (latents_temp - (1-self.scheduler.alphas_cumprod[t])**0.5 * noise_pred)

                            # 2. decode x0 with vae decoder
                            # 오디오 잠재 공간에에서 멜 스펙트로그램으로 디코딩
                            x0_mel_spectrogram = self.decode_latents(x0)

                            if x0_mel_spectrogram.dim() == 4:
                                x0_mel_spectrogram = x0_mel_spectrogram.squeeze(1)

                            # 3. convert mel-spectrogram to waveform
                            # 멜 스펙트로그램을 파형으로 변환
                            x0_waveform = self.vocoder(x0_mel_spectrogram)

                            # 4. waveform to imagebind mel-spectrogram
                            x0_imagebind_audio_input = load_and_transform_audio_data_from_waveform(x0_waveform, org_sample_rate=self.vocoder.config.sampling_rate,
                                                device=device, target_length=204, clip_duration=clip_duration, clips_per_video=clips_per_video)

                            # compute loss with imagebind
                            # ImageBind 모델을 통해 텍스트, 비디오, 오디오의 임베딩 추출
                            if isinstance(prompt, str):
                                prompt_bind = [prompt]
                            inputs = {
                                ModalityType.VISION: image_bind_video_input,
                                ModalityType.AUDIO: x0_imagebind_audio_input,
                                ModalityType.TEXT: load_and_transform_text(prompt_bind, device)
                            }

                            # with torch.no_grad():
                            embeddings = bind_model(inputs)
                            # 듀얼/삼각 손실 함수의 거리 계산
                            # F(e_a, e_p) : 텍스트-오디오 임베딩 거리
                            bind_loss_text_audio = 1 - F.cosine_similarity(embeddings[ModalityType.TEXT], embeddings[ModalityType.AUDIO])
                            # F(e_v, e_a) : 비디오-오디오 임베딩 거리
                            bind_loss_vision_audio = 1 - F.cosine_similarity(embeddings[ModalityType.VISION], embeddings[ModalityType.AUDIO])
                            # 최종 정렬 손실
                            bind_loss = bind_loss_text_audio + bind_loss_vision_audio

                            bind_loss.backward()
                            optimizer.step()
                            optimizer.zero_grad()

                latents = latents_temp.detach() # 업데이트된 잠재변수를 다음 denoising 단계로 전달

                # call the callback, if provided
                if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) % self.scheduler.order == 0):
                    progress_bar.update()
                    if callback is not None and i % callback_steps == 0:
                        callback(i, t, latents)

        # 8. Post-processing
        # 최종 잠재 변수 z_0을 멜 스펙트로그램, 파형으로 변환하여 오디오 생성 완료
        mel_spectrogram = self.decode_latents(latents)

        audio = self.mel_spectrogram_to_waveform(mel_spectrogram) # [1, 128032]
        audio = audio[:, :original_waveform_length] # [1, 128000]

        if output_type == "np":
            # audio = audio.numpy()
            audio = audio.detach().numpy()

        if not return_dict:
            return (audio,)

        return AudioPipelineOutput(audios=audio)